#tinyMeteo - weather forecasting on microcontroller

## Collect Data 

Collect weather data for years 2009-2022 for the desired location from World Weather Online API in JSON format.

In [85]:
!rm -r parsed_weather_data
!rm weather_data.csv
!rm weather_data_parsed.csv
!rm final_data.csv

rm: cannot remove 'parsed_weather_data': No such file or directory
rm: cannot remove 'weather_data.csv': No such file or directory
rm: cannot remove 'weather_data_parsed.csv': No such file or directory


In [ ]:
from google.colab import files 
files.upload() #upload a .json file with Kaggle API key and location coordinates or location name

In [2]:
import json

# Read the uploaded files for the API key and location detials:
with open('api.json') as f:
    api_details = json.load(f)

# Access the API key and location values
API_KEY = api_details['key']
LOCATION = api_details['location']

In [3]:
!rm api.json # deletd .json file

### API calls
API only allows 30days of data per call. Loop through API calls for all necessary data and all .json files for each each month in a 'weather_data' directory.

In [4]:
import os
import requests
import json
from calendar import monthrange

start_year = 2009
end_year = 2022

# path of raw data
raw_data = 'weather_data'

# Create the 'weather_data' directory if it does not already exist
if not os.path.exists(raw_data):
    os.makedirs(raw_data)

#     # Loop through all years and months and create a file for each month
for year in range(start_year, end_year+1):
    for month in range(1, 13):
        # Get the start and end dates for the month
        _, num_days = monthrange(year, month) 
        start_date = f'{year}-{month:02d}-01'
        end_date = f'{year}-{month:02d}-{num_days:02d}'
        
        # Construct the URL and make the request
        url = f'https://api.worldweatheronline.com/premium/v1/past-weather.ashx?key={API_KEY}&q={LOCATION}&format=json&date={start_date}&enddate={end_date}'
        response = requests.get(url)
        data = response.json()
        
        # Get the filename and filepath for this month's data
        filename = f'weather_{year}-{month:02d}.json'
        filepath = os.path.join(raw_data, filename)
        
        # Check if file exists and create it if it does not
        if not os.path.exists(filepath):
            with open(filepath, 'w') as f:
                json.dump(data, f)
        else:
            print(f'File {filename} already exists.')


### Filter JSON

Filter all created .json files for the date, time, temperature, humidity, pressure, precipitation, and weather description keys and create new parsed .json files in a new directory called 'parsed_weather_data'.

In [5]:
parsed_data_path = 'parsed_weather_data'

# create the directory for the parsed weather data if it doesn't exist
if not os.path.exists(parsed_data_path):
    os.makedirs(parsed_data_path)

# loop through all the JSON files in the weather_data directory
for filename in os.listdir(raw_data):
    if filename.endswith('.json'):
        # extract the location and month from the filename
        location_name = filename.split('_')[0]
        month = filename.split('_')[1].split('.')[0]

        # create a new dictionary to store the parsed data for this month
        parsed_month_data = {}

        # Open the file and load the JSON data
        with open(f'{raw_data}/{filename}', 'r') as f:
            file_contents = f.read()
            parsed_data = json.loads(file_contents)

        # loop through the hourly data for all the dates in the JSON file
        for weather_data in parsed_data['data']['weather']:
            # extract the date for this set of hourly data
            date = weather_data['date']

            # create a new list to store the parsed data for this day
            parsed_day_data = []

            # loop through the hourly data for this date
            for hourly_data in weather_data['hourly']:
                # extract the values we're interested in
                tempC = hourly_data['tempC']
                humidity = hourly_data['humidity']
                pressure = hourly_data['pressure']
                weatherCond = hourly_data['weatherDesc'][0]['value']
                time = hourly_data['time']

                # add the values to the parsed_day_data list
                parsed_day_data.append({
                    'time': time,
                    'tempC': tempC,
                    'humidity': humidity,
                    'pressure': pressure,
                    'weatherCond': weatherCond
                })

            # add the parsed day data to the parsed_month_data dictionary
            parsed_month_data[date] = parsed_day_data

        # write the parsed_month_data to a new JSON file
        with open(f'{parsed_data_path}/{location_name}_{month}_parsed.json', 'w') as f:
            json.dump(parsed_month_data, f)


Combine all parsed .json files into a single .json file containing all data of interest.

In [6]:
# create the combined data dictionary
combined_data = {}

# combined parsed json data path
json_path = f'{parsed_data_path}/_combined_parsed_data.json'

# loop over each file in the directory
for filename in os.listdir(parsed_data_path):
    if filename.endswith('.json'):
        # load the contents of the file into a dictionary
        with open(os.path.join(parsed_data_path, filename)) as f:
            parsed_data = json.load(f)
        
        # loop over each date in the parsed data dictionary
        for date in parsed_data.keys():
            # if the date doesn't exist in the combined data dictionary, add it with an empty list as its value
            if date not in combined_data:
                combined_data[date] = []
            
            # add the weather data to the combined_data dictionary
            combined_data[date].extend(parsed_data[date])

# save the combined data to a new file
with open(json_path, 'w') as f:
    json.dump(combined_data, f)


### Convert to CSV

In [8]:
import csv
from datetime import datetime

# the csv that contains all values
csv_data_file = '_combined_parsed_data.csv'
# the parsed csv in the desired final format
final_file = "final_data.csv"

if not os.path.isfile(f'{parsed_data_path}/{csv_data_file}'):
    with open(f'{parsed_data_path}/{csv_data_file}', "x") as f:
        pass

# Write initial CSV file with headers and weather data
with open(f'{parsed_data_path}/{csv_data_file}', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['date', 'time', 'tempC', 'humidity', 'pressure', 'weatherCond'])

    with open(json_path, 'r') as f:
        weather_data = json.load(f)

    for date in weather_data:
        for data in weather_data[date]:
            writer.writerow([date, data['time'], data['tempC'], data['humidity'], data['pressure'], data['weatherCond']])

# Read the initial CSV file, transform the date format, and write to the final file
with open(f'{parsed_data_path}/{csv_data_file}', 'r') as infile, open(final_file, 'w', newline='') as outfile:
    reader = csv.DictReader(infile)
    fieldnames = ['date', 'year', 'month', 'day', 'time', 'tempC', 'humidity', 'pressure', 'weatherCond']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        date = datetime.strptime(row['date'], '%Y-%m-%d').strftime('%Y%m%d')
        year, month, day = date[:4], date[4:6], date[6:]
        writer.writerow({
            'date': date,
            'year': year,
            'month': month,
            'day': day,
            'time': row['time'],
            'tempC': row['tempC'],
            'humidity': row['humidity'],
            'pressure': row['pressure'],
            'weatherCond': row['weatherCond']
        })


In [ ]:
# install pandas module
!pip install pandas

## Preprocessing of data

In [91]:
import pandas as pd

# read csv file
df = pd.read_csv('final_data.csv', sep=',') 
#show the first five rows
df.head()

,date,year,month,day,time,tempC,humidity,pressure,weatherCond
0,20210601,2021,6,1,0,19,74,1011,Patchy rain possible
1,20210601,2021,6,1,100,18,76,1011,Cloudy
2,20210601,2021,6,1,200,18,77,1011,Cloudy
3,20210601,2021,6,1,300,17,78,1011,Cloudy
4,20210601,2021,6,1,400,18,77,1010,Cloudy


In [92]:
# select only the values we need\
df = df[['tempC', 'humidity','pressure','weatherCond']]

In [53]:
df.shape

(122712, 4)

In [14]:
df.describe()

,humidity,pressure
count,122712.000000,122712.000000
mean,62.035416,1014.878692
std,14.964368,6.113101
min,17.000000,984.000000
25%,51.000000,1011.000000
50%,62.000000,1014.000000
75%,74.000000,1019.000000
max,99.000000,1039.000000


In [15]:
df.values

array([[74, 1011, 'Patchy rain possible'],
       [76, 1011, 'Cloudy'],
       [77, 1011, 'Cloudy'],
       ...,
       [46, 1021, 'Clear'],
       [49, 1021, 'Clear'],
       [53, 1022, 'Partly cloudy']], dtype=object)

In [16]:
df.info

<bound method DataFrame.info of         humidity  pressure           weatherCond
0             74      1011  Patchy rain possible
1             76      1011                Cloudy
2             77      1011                Cloudy
3             78      1011                Cloudy
4             77      1010                Cloudy
...          ...       ...                   ...
122707        40      1020                 Clear
122708        43      1021                 Clear
122709        46      1021                 Clear
122710        49      1021                 Clear
122711        53      1022         Partly cloudy

[122712 rows x 3 columns]>

### Assign Labels
Assign numbers to weather description values

In [93]:
# Assign an integer value to each weather description in the "weather description" column
df = df.dropna()  # remove empty rows

# Convert each category into an integer
for i in df.index:
    if df.loc[i, "weatherCond"] == 'Clear':
        df.loc[i, "weatherCond"] = 0
    elif df.loc[i, "weatherCond"] == 'Sunny':
        df.loc[i, "weatherCond"] = 0
    elif df.loc[i, "weatherCond"] == 'Partly cloudy':
        df.loc[i, "weatherCond"] = 1
    elif df.loc[i, "weatherCond"] == 'Cloudy':
        df.loc[i, "weatherCond"] = 1
    elif df.loc[i, "weatherCond"] == 'Overcast':
        df.loc[i, "weatherCond"] = 1
    elif df.loc[i, "weatherCond"] == 'Moderate rain at times':
        df.loc[i, "weatherCond"] = 2
    elif df.loc[i, "weatherCond"] == 'Patchy rain possible':
        df.loc[i, "weatherCond"] = 2
    elif df.loc[i, "weatherCond"] == 'Moderate or heavy rain shower':
        df.loc[i, "weatherCond"] = 3
    elif df.loc[i, "weatherCond"] == 'Heavy rain at times':
        df.loc[i, "weatherCond"] = 3
    elif df.loc[i, "weatherCond"] == 'Light freezing rain':
        df.loc[i, "weatherCond"] = 4
    elif df.loc[i, "weatherCond"] == 'Patchy moderate snow':
        df.loc[i, "weatherCond"] = 4
    else:
        df = df.drop([i])  # we don't consider other classes, so we drop them

# Cast weatherCond column from string to int
df["weatherCond"] = df["weatherCond"].astype(int)


### Case1: Decision Tree Classifier

In [94]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

X = df.drop(columns=['weatherCond'])
y = df['weatherCond']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = DecisionTreeClassifier()
model.fit(X_train,y_train)
predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score

0.6907875972782463

### Case 2: Dense Neural Network

### Split Data

In [86]:
import numpy as np
from keras.utils import to_categorical

labels = df.pop('weatherCond').values # Create labels as integer array
features = df.values # Use DataFrame directly as features

In [43]:
features

array([[  74, 1011],
       [  76, 1011],
       [  77, 1011],
       ...,
       [  46, 1021],
       [  49, 1021],
       [  53, 1022]])

In [44]:
labels

array([[0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.]], dtype=float32)

In [87]:
from sklearn.model_selection import train_test_split

#Split the dataset into training set 85% and test set 15%
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.20, shuffle=True)



## The Model

The model used is a densely connected neural network (DNN). The activation function used is ReLU (rectified linear unit), and the output layer uses the softmax activation function. The dropout regularization technique is applied with a rate of 0.4, which randomly drops connections between neurons to prevent overfitting.

The model is compiled using the Adam optimizer, categorical cross-entropy loss function, and accuracy metric. The summary method is called to display the model's architecture and parameters.

In [88]:
import tensorflow as tf
from tensorflow.keras import regularizers
from keras.utils import to_categorical

# Parameters
NB_classes = 5 # Number of outputs
NB_neurones = 3 # Main number of neurons
NB_features = 3 # Number of inputs
activation_func = tf.keras.activations.relu # Activation function used

# One-hot encode target labels
train_labels = to_categorical(train_labels, num_classes=NB_classes)
test_labels = to_categorical(test_labels, num_classes=NB_classes)

#Densly connected neural network
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func,input_shape=(NB_features,)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dense(NB_neurones,activation=activation_func, kernel_regularizer=regularizers.l2(0.01)),
                             tf.keras.layers.Dropout(0.4), #drop randomly some connection to avoid overfiting
                             #softmax will output an array containing probabilities of each classes
                             #the highest one is the predicted class
                             tf.keras.layers.Dense(NB_classes,activation=tf.keras.activations.softmax)
])

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer=custom_optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 3)                 12        
                                                                 
 dense_118 (Dense)           (None, 3)                 12        
                                                                 
 dense_119 (Dense)           (None, 3)                 12        
                                                                 
 dense_120 (Dense)           (None, 3)                 12        
                                                                 
 dense_121 (Dense)           (None, 3)                 12        
                                                                 
 dense_122 (Dense)           (None, 3)                 12        
                                                                 
 dense_123 (Dense)           (None, 3)                

## Train the Model

In [89]:
model.fit(x=train_features,
          y=train_labels,
          epochs=20,
          validation_data=(test_features, test_labels),
          verbose=1,
          shuffle=True)

Epoch 1/20
3068/3068 [==============================] - 15s 4ms/step - loss: 1.6716 - accuracy: 0.6804 - val_loss: 1.4605 - val_accuracy: 0.6820
Epoch 2/20
3068/3068 [==============================] - 12s 4ms/step - loss: 1.3113 - accuracy: 0.6804 - val_loss: 1.1851 - val_accuracy: 0.6820
Epoch 3/20
3068/3068 [==============================] - 12s 4ms/step - loss: 1.1000 - accuracy: 0.6804 - val_loss: 1.0290 - val_accuracy: 0.6820
Epoch 4/20
3068/3068 [==============================] - 11s 4ms/step - loss: 0.9847 - accuracy: 0.6804 - val_loss: 0.9467 - val_accuracy: 0.6820
Epoch 5/20
3068/3068 [==============================] - 11s 4ms/step - loss: 0.9246 - accuracy: 0.6804 - val_loss: 0.9036 - val_accuracy: 0.6820
Epoch 6/20
3068/3068 [==============================] - 12s 4ms/step - loss: 0.8929 - accuracy: 0.6804 - val_loss: 0.8806 - val_accuracy: 0.6820
Epoch 7/20
3068/3068 [==============================] - 12s 4ms/step - loss: 0.8758 - accuracy: 0.6804 - val_loss: 0.8676 - val_ac

## Evaluate

In [90]:
performance=model.evaluate(test_features,test_labels, batch_size=32, verbose=1, steps=None, )[1] * 100
print('Final accuracy : ', round(performance), '%')

767/767 [==============================] - 1s 2ms/step - loss: 0.8430 - accuracy: 0.6820
Final accuracy :  68 %
